# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Целль исследования** - проверить гепотезы:
1. Наличие детей влияет на возврат кредита в срок.
2. Семейное положение влияет на возврат кредита в срок.
3. Уровень дохода влияет на возврат кредита в срок.
4. Цель кредита влиет на возврат кредита в срок.

**Ход исследования**

Данные о кредитах получим из файла `data.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

Проверим данные на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки исправим ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Прочитаем файл data.csv из папки /datasets и сохраним его в переменной df и выведем на экран первые десять строк таблицы.

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Одной командой получим общую информацию о таблице:

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице пятнадцать столбцов. Типы данных в столбцах — `float`, `int`, `object`.

Согласно документации к данным:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

В данных встречаются два типа числовых данных, что может создать проблемы во время исследования.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

В столбце `education` встречаются одни и те же слова в разных регистрах.

**Вывод**

В каждой строке таблицы — данные о клиентах банка. Колонка с целью получения описывает кредит. Остальные данные рассказывают о самом клиенте.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных и два типа числовых значений.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

Сначала посчитаем, сколько в таблице пропущенных значений.

In [3]:
display(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Данные пропущены в столбцах `days_employed` и `total_income`. В `days_employed` данные могут быть пропущены по двум причинам: клиент не работал или неизвестно количество проработанных дней. Что насчёт `total_income`, то тут клиент либо не работает, либо нет данных по его доходу, либо нет фиксированного дохода.

Не все пропущенные значения влияют на исследование. Так в `days_employed` пропуски не важны для работы. Поэтому доостаточно заменить пропуски явными обозначениями.

Заменим пропущенные значения в столбце `days_employed` на строку `'unknown'`. 


In [4]:
df['days_employed'] = df['days_employed'].fillna('unknown')

Заменим пропущенные значения в столбце `total_income` на медианные значения доходов в зависимости от типа занятости.

Посмотрим по каким типам занятости есть пропуски в доходах.

In [5]:
df[df.isna().any(axis=1)].groupby('income_type')['children'].count().rename("count")

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: count, dtype: int64

Пропуски в доходах есть у госслужащих, компаньонов, пенсионеров, предпринимателей и у сотрудников.

Напишем функцию, которая найдёт медиану доходов по заданному значению столбца `income_type`. Вызовем функцию для каждого типа занятости, где есть пропуски в доходах, и заполним пропуски найденными медианами.

In [6]:
def income_median(type):
    values = df[df['income_type'] == type][['total_income']].dropna()
    median = values['total_income'].sort_values(ascending=True).median()
    return median
    
income_types = ['госслужащий', 'компаньон', 'пенсионер', 'предприниматель', 'сотрудник']
for word in income_types:
    df['total_income'] = df['total_income'].fillna(income_median(word))

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения.

In [7]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Обработка пропусков обнаружила пропуски в двух колонках.

Мы выяснили, что значения в `days_employed` не важны для работы, поэтому заминили пропущенные значения на `'unknown'`. А вот значения в столбце `days_employed` нам нужны, поэтому решили проблему с пропусками, заменив их на медианное значение столбца.

Перейдём к замене типов данных

### Замена типа данных

Для последующего анализа нам пригодятся числовые данные, однако некоторые из нужных нам столбцов находятся в разных типах данных, что помешает в работе. 

Переведём тип данных столбца `total_income` к `int`.

Для изменения типа данных нам подойдёт метод `.astype()`, так как данный метод работает с `Series` и позволяет изменить тип данных на `int`.

In [8]:
df['total_income'] = df['total_income'].astype('int')

Проверим результат, выведем основную информацию о датасете.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null object
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(6), object(6)
memory usage: 2.0+ MB


**Вывод**

Мы изменили тип данных в столбце `total_income` методом `.astype()`. Приступим к обработке дубликатов.

### Обработка дубликатов

Чтобы учесть все дубликаты, приведём все символы в строке `education` к нижнему регистру. И посмотрим уникальные значения столбца, чтобы убедиться в работе кода и проверить нет ли ещё проблем в данных.

In [10]:
df['education'] = df['education'].str.lower()
print(df['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


Тепреь посчитаем количество явных дубликатов для датасета.

In [11]:
df.duplicated().sum()

71

Повтор в данных мог возникнуть из за столбца `education`, и вводились одинаковые данные, но с разным регистром оного и того же слова.

Удалим явные дубликаты и убедимся, что всё было сделано правильно.

In [12]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

**Вывод**

Избавились от дубликатов в данных, предварительно преведя строки к нижнему регистру.

### Лемматизация

Посмотрим уникальные значения столбца `purpose`, чтобы понять какие данные надо будет лемматизировать.

In [13]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

В целях кредита можно выделить несколько основных слов: жилье, недвижимость, образование, автомобиль, свадьба. Так как жильё и невижимость слова синонимы, выделим общую часть из них - 'жил'. 

Создадим новый стобец `purpose_short`, в котором будут указаны цели кредита.
Для этого создадим пустой список `purposes` и цикл, который обработает столбец с целями кредита и добавит лемматизированные цели в спиоск `purposes`. 
В целях кредита можно выделить несколько основных слов: жилье, недвижимость, образование, автомобиль, свадьба. Так как жильё и невижимость слова синонимы, выделим общую часть из них - 'жил'. В словах образование и автомобиль выделим корни 'образов' и 'авто', остальные цели отнесём к 'свадьбе', так как иных целей больше нет.
Добавим получившийся список к датафрейму.

In [14]:
purposes = []
for row in df['purpose']:
    if ('жил' in row) or ('недвиж' in row):
        purposes.append('недвижимость')
    elif 'образован' in row:
        purposes.append('образование')
    elif 'авто' in row:
        purposes.append('автомобиль')
    else:
        purposes.append('свадьба')

df['purpose_short'] = purposes
df['purpose_short']

0        недвижимость
1          автомобиль
2        недвижимость
3         образование
4             свадьба
             ...     
21449    недвижимость
21450      автомобиль
21451    недвижимость
21452      автомобиль
21453      автомобиль
Name: purpose_short, Length: 21454, dtype: object

**Вывод**

Мы лемматизировали данные и добавили их в новый стобец, теперь можно приступать к последнему шагу предобработки данных - категоризации данных.

<div  style="height:90px; background:#EEEEFF; padding: 15px;">
<font size="5"><b>Коммент от студента без тайменеджмента</b></font>

Добавила описание выбора лемм.
</div>

### Категоризация данных

Категоризируем данные по ежемесячному доходу.  Добавим новый столбец в котором будет указан уровень дохода.

Создадим функцию, которая обработает данные о ежемесячном доходе. Основываясь на средних зарплатах по Москве, выделим категории, если доход меньше или равен 80 000, тогда функция вернёт 'ниже среднего', если доход меньше или равен 150 000, вернёт 'средний доход', если больше 150 000, тогда 'выше среднего'.

In [15]:
def income_lvl(income):
    if income <= 80000:
        return 'ниже среднего'
    if income <= 150000:
        return 'средний доход'
    return 'выше среднего'

df['income_lvl'] = df['total_income'].apply(income_lvl)
df['income_lvl'].value_counts()

выше среднего    11287
средний доход     7891
ниже среднего     2276
Name: income_lvl, dtype: int64

Получились группы достаточного размера для анализа. Данные категоризированны.

Выделим два словаря `education_dict` и `family_status_dict`.

In [16]:
df.groupby('education_id')['education'].unique()

education_id
0                 [высшее]
1                [среднее]
2    [неоконченное высшее]
3              [начальное]
4         [ученая степень]
Name: education, dtype: object

In [17]:
df.groupby('family_status_id')['family_status'].unique()

family_status_id
0          [женат / замужем]
1         [гражданский брак]
2           [вдовец / вдова]
3                [в разводе]
4    [Не женат / не замужем]
Name: family_status, dtype: object

In [18]:
education_dict = {0: 'высшее',
                  1: 'среднее',
                  2: 'неоконченное высшее',
                  3: 'начальное',
                  4: 'ученая степень'}

family_status_dict = {0: 'женат / замужем',
                      1: 'среднее',
                      2: 'вдовец / вдова',
                      3: 'в разводе',
                      4: 'Не женат / не замужем'}

**Вывод**

Мы категоризировали данные по ежемесячному доходу, а так же создали два словаря: `education_dict` и `family_status_dict`. Данные готовы к тому, чтобы дать ответ на поставленные вопрсы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [19]:
df = df.replace({'children':{-1:1}})
df_children = df.groupby(by=['children'])['debt'].agg(['count','sum'])
df_children['conversion'] = df_children['sum'] / df_children['count']
display(df_children)

conversion_children = df_children['conversion'].mean()
print('Средняя конверсия {:.4f}'.format(conversion_children))

,count,sum,conversion
children,,,
0,14091,1063,0.075438
1,4855,445,0.091658
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000
20,76,8,0.105263


Средняя конверсия 0.0780


**Вывод**

Наибольшая конверсия в семьях, где есть 1-2 ребёнка, значит чаще всего задолженности имеют такие семьи, примерно 9% таких семей от семей с таким же количеством детей, имеют задолженности. Наименьшую часть задолжностей имеют семьи без детей, они чаще всех укладываются в срок оплаты. Так же высокую конверсию можно заметить у семей с 4 детьми, однако данных по таким семьям довольно таки мало, что не позволяет сделать точный вывод о ситуации с задолженностями. То же самое можно сказать и о семьях с 5 и 20 детьми, хоть и конверссия очень низкая и даже отсутствует, что означает что такие семьи не имеют задолженностей, однако данных тоже мало, чтобы делать какие-либо выводы.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [20]:
df_family_status = df.groupby(by=['family_status'])['debt'].agg(['count','sum'])
df_family_status['conversion'] = df_family_status['sum'] / df_family_status['count']
display(df_family_status)

conversion_family = df_family_status['conversion'].mean()
print('Средняя конверсия {:.4f}'.format(conversion_family))

,count,sum,conversion
family_status,,,
Не женат / не замужем,2810,274,0.097509
в разводе,1195,85,0.071130
вдовец / вдова,959,63,0.065693
гражданский брак,4151,388,0.093471
женат / замужем,12339,931,0.075452


Средняя конверсия 0.0807


**Вывод**

Чаще всего задолженности имеют люди не состоящие в браке или находящиеся в гражданском браке. Реже всего задолжниками становятся вдовы и вдовцы.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [21]:
df_income_lvl = df.groupby(by=['income_lvl'])['debt'].agg(['count','sum'])
df_income_lvl['conversion'] = df_income_lvl['sum'] / df_income_lvl['count']
display(df_income_lvl)

conversion_income = df_income_lvl['conversion'].mean()
print('Средняя конверсия {:.4f}'.format(conversion_income))

,count,sum,conversion
income_lvl,,,
выше среднего,11287,896,0.079383
ниже среднего,2276,174,0.076450
средний доход,7891,671,0.085034


Средняя конверсия 0.0803


**Вывод**

Судя по конверсии, задолжености чаще имеют люди со средним доходом, и реже всего с доходом выше и ниже среднего.

- Как разные цели кредита влияют на его возврат в срок?

In [22]:
df_purpose_short = pd.pivot_table(df, index='purpose_short', values='debt', aggfunc=[len,sum])
df_purpose_short['conversion'] = df_purpose_short['sum']['debt'] / df_purpose_short['len']['debt']
display(df_purpose_short)

conversion_purpose_short = df_purpose_short['conversion'].mean()
print('Средняя конверсия {:.4f}'.format(conversion_purpose_short))

,len,sum,conversion
,debt,debt,
purpose_short,,,
автомобиль,4306,403,0.093590
недвижимость,10811,782,0.072334
образование,4013,370,0.092200
свадьба,2324,186,0.080034


Средняя конверсия 0.0845


**Вывод**

Если смотреть по связи между целю кредита и задолженностями, то тут наиболее сильная связь с такими целями, как автомобиль и образование. Наименьшую связь с задолженностями имеют кредиты с целью недвижемости.

## Шаг 4. Общий вывод

Мы проверили четыре гипотезы и установили:

1. Зависимость между наличием детей и возвратом кредита в срок есть. Люди без детей чаще оплачивают кредит в срок.

Первая гипотеза подтвердилась.

2. Зависимость между семейным положением и возвратом кредита в срок есть. Зависимость здесь наименьшая, но она присутствует. По данному критерию можно сказать, что вдовы и вдовцы реже всего становятся задолжниками.

Таким образом, вторая гипотеза тожевердилась.

3. Зависимость между уровнем дохода и возвратом кредита в срок так же присутствует. Меньше всего задолжников с доходом выше и ниже среднего.

Третья гипотеза подтвердилась.

4. Разные цели кредита влияют на его возврат в срок. Наименьшую связь с задолженностями имеют кредиты с целью недвижемости.

Четвертая гепотиза тоже подтвердилась.

Каждый из факторов влияет на возврат кредита в срок. Сильнее всего влияет количество детей и цели кредита. Однако, разница в значениях не сильно велика, поэтому выделить конкретный критерий, по которому можно судить о возврате кредита в срок,нельзя.